In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import preproccessing as prep
from evaluation import show_scenarios_evaluation



In [3]:
acwi = pd.read_excel('acwi_usd_net_31.11.23.xlsx',index_col='Date', parse_dates=True)
acwi = acwi.squeeze()
acwi = (acwi+1).cumprod()
acwi.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 430 entries, 1988-01-29 to 2023-10-31
Series name: ACWI
Non-Null Count  Dtype  
--------------  -----  
430 non-null    float64
dtypes: float64(1)
memory usage: 6.7 KB


In [8]:
usdpln = prep.loadStooqData('usdpln','2009-12-01','d')
usdpln = usdpln.resample('BM').last()
usdpln_ret = prep.log_returns(usdpln)
acwi_pln = (acwi * usdpln).dropna()
acwi_pln_returns = prep.log_returns(acwi_pln)

In [14]:
wig = prep.loadStooqData('MWIG40TR','2009-12-01',frequency='d')
wig = wig.resample('BM').last()[:'2023-10-31']
wig_returns = prep.log_returns(wig)

In [15]:
acwi_pln_returns.index.shape

(166,)

In [16]:
wig_returns.index.shape

(166,)

In [20]:
exog = sm.add_constant(wig_returns)
model = sm.regression.linear_model.OLS(endog=acwi_pln_returns,exog=exog)
res = model.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.172
Model:                            OLS   Adj. R-squared:                  0.167
Method:                 Least Squares   F-statistic:                     34.13
Date:                Sun, 10 Dec 2023   Prob (F-statistic):           2.70e-08
Time:                        22:26:08   Log-Likelihood:                 347.75
No. Observations:                 166   AIC:                            -691.5
Df Residuals:                     164   BIC:                            -685.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0066      0.002      2.821      0.005       0.002       0.011
Zamkniecie     0.2691      0.046      5.842      0.000       0.178       0.360
==============================================================================
Omnibus:                        2.227   Durbin-Watson:                   1.951
Prob(Omnibus):                  0.328   Jarque-Bera (JB):                2.077
Skew:                           0.022   Prob(JB):                        0.354
Kurtosis:                       3.546   Cond. No.                         19.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
exog = sm.add_constant(acwi_pln_returns)
model = sm.regression.linear_model.OLS(endog=wig_returns,exog=exog)
res = model.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Zamkniecie   R-squared:                       0.172
Model:                            OLS   Adj. R-squared:                  0.167
Method:                 Least Squares   F-statistic:                     34.13
Date:                Sun, 10 Dec 2023   Prob (F-statistic):           2.70e-08
Time:                        22:28:45   Log-Likelihood:                 275.83
No. Observations:                 166   AIC:                            -547.7
Df Residuals:                     164   BIC:                            -541.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0016      0.004      0.420      0.675      -0.006       0.009
0              0.6401      0.110      5.842      0.000       0.424       0.856
==============================================================================
Omnibus:                        5.896   Durbin-Watson:                   1.781
Prob(Omnibus):                  0.052   Jarque-Bera (JB):                6.148
Skew:                          -0.307   Prob(JB):                       0.0462
Kurtosis:                       3.715   Cond. No.                         30.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""